In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import re
allLines = []

def getInputFile(filename):
    if(filename[len(filename)-4:len(filename)] != ".ann"):
        print("Invalid File")
        getInputFile()
    else:
        file = open(filename, "r")
        
def formatText(str):
    print(str)
    for strings in str[2].split():
        if(str[2].split().index(strings) == 0):
            if(len((str[2].split())) == 1):
                f.write(strings+"\t"+"O-"+str[1].split()[0])
                f.write("\n")
            else:
                f.write(strings+"\t"+"B-"+str[1].split()[0])
                f.write("\n")
        else:
            f.write(strings+"\t"+"I-"+str[1].split()[0])
            f.write("\n")
    return

from glob import glob

ann_dir = glob("/home/taille/these/data/QUAERO_FrenchMed_brat/QUAERO_FrenchMed/corpus/dev/MEDLINE/*.ann")

getInputFile(input_filename)
for line in file:
    allLines.append(re.split(r'\t+', line.rstrip('\t')))
sortedLines = sorted(allLines,key=lambda x: x[1].split()[1])
sortedLinesFinal = sorted(allLines,key=lambda x: x[1].split()[2])

f = open(outputFilename+".txt","a+")
f.write('\n')
for eachEntity in sortedLinesFinal:
    formatText(eachEntity)
f.close()

In [3]:
import pandas as pd

df = pd.read_csv('/home/taille/these/data/ncbi_conll/ncbi_conll_train_4col.csv', sep=' ')

df = df.replace('', np.nan)

df['O'].value_counts()

O                     98457
I-SpecificDisease      3104
B-SpecificDisease      2222
B-Modifier              963
I-DiseaseClass          704
I-Modifier              670
B-DiseaseClass          559
I-CompositeMention      429
B-CompositeMention       94
Name: O, dtype: int64

In [258]:
# remove all spaces between '.'
cas_file = '/home/taille/these/data/corpus_dalloux/cas_conll_train.csv'

with open(cas_file, 'r') as f:
    lines = f.readlines()

sentences = ''.join(lines[2:]).split('. NN O O\n')

new_lines = lines[:2]

for s in sentences:
    lines = s.strip('\n').split('\n')
    for l in lines:
        if l.strip()!='' and len(l.split(' ')) > 1:
            new_lines.append(l)
        elif len(l.split(' ')) <=1:
            new_lines.append(l + ' NN O O')
    new_lines.append('. NN O O\n')

with open(cas_file.replace('.csv', '_4col.csv'), 'w') as f:
    f.write('\n'.join(new_lines))

In [5]:
docstart = '-DOCSTART- -X- -X- O\n'

ncbi_dir = '/home/taille/these/data/ncbi_conll/small_files/'

from glob import glob

ncbi_files = glob(ncbi_dir + '*.conll')

ncbi_df_list = list(map(lambda x: pd.read_csv(x[1], sep='\t', header=None).drop(columns=0).assign(doc_id=x[0]), enumerate(ncbi_files)))

ncbi_df = pd.concat(ncbi_df_list).rename(columns={1: 'word', 2: 'ner_tag'})[['doc_id', 'word', 'ner_tag']]
ncbi_df['word'] = ncbi_df['word'].str.strip()
ncbi_df['ner_tag'] = ncbi_df['ner_tag'].str.strip()
ncbi_df.loc[ncbi_df.word=='${newline}', ['word', 'ner_tag']] = ''

from sklearn.model_selection import train_test_split

train_ids, test_ids = train_test_split(ncbi_df['doc_id'].unique())
train_ids, val_ids = train_test_split(train_ids)

train_df = ncbi_df[ncbi_df.doc_id.isin(train_ids)]
val_df = ncbi_df[ncbi_df.doc_id.isin(val_ids)]
test_df = ncbi_df[ncbi_df.doc_id.isin(test_ids)]

# train_df[['word', 'ner_tag']].to_csv('/home/taille/these/data/ncbi_conll/ncbi_conll_train.csv', index=False, sep=' ')
# val_df[['word', 'ner_tag']].to_csv('/home/taille/these/data/ncbi_conll/ncbi_conll_val.csv', index=False, sep=' ')
# test_df[['word', 'ner_tag']].to_csv('/home/taille/these/data/ncbi_conll/ncbi_conll_test.csv', index=False, sep=' ')

ds = pd.DataFrame([['-DOCSTART-', 'O']], columns=['word','ner_tag'])

def add_row(x):
    return x.append(ds)

# train_df = train_df.groupby('doc_id').apply(add_row).reset_index(drop=True)
# test_df = test_df.groupby('doc_id').apply(add_row).reset_index(drop=True)
# val_df = val_df.groupby('doc_id').apply(add_row).reset_index(drop=True)

,doc_id,word,ner_tag
0,1,Familial,B-SpecificDisease
1,1,deficiency,I-SpecificDisease
2,1,of,I-SpecificDisease
3,1,the,I-SpecificDisease
4,1,seventh,I-SpecificDisease
...,...,...,...
271,791,patient,O
272,791,with,O
273,791,FHM,B-SpecificDisease
274,791,.,O


In [27]:
# ADD DOCSTART BETWEEN DOCUMENTS

conll_files = [
    "/home/taille/these/data/ncbi_conll/ncbi_conll_train.csv",
    "/home/taille/these/data/ncbi_conll/ncbi_conll_val.csv",
    "/home/taille/these/data/ncbi_conll/ncbi_conll_test.csv"
]

for conll_file in conll_files:
    with open(conll_file, 'r') as f:
        lines = ''.join(f.readlines())

    lines = lines.replace('. O\n. O\n', '. O\n')

    lines = lines.split('\n')

    docstart_str = '-DOCSTART- -X- -X- O\n\n'
    
    new_lines = [docstart_str]
    for i, l in enumerate(lines[1:]):
        if (l != '\n') and (l!=' \n') and (l!=' ') and (l!='') and (l!='. O'):
            line = l.split(' ')
            new_line = line[0] + ' NN O ' + line[-1] + '\n'
            new_lines.append(new_line)
        elif l==' ':
            if lines[i]=='. O':
                new_lines.append(docstart_str)
        elif l=='. O':
            new_lines.append('. NN O O\n\n')
        else:
            new_lines.append('\n')

    new_conll_file = conll_file.replace('.csv', '_4col.csv')

    with open(new_conll_file, 'w') as f:
        f.write(''.join(new_lines))
        print('written')


written
written
written


In [176]:
import pandas as pd

conll_file = "/home/taille/these/data/ncbi-disease-master/conll/train.tsv"

with open(conll_file, 'r') as f:
    lines = f.readlines()
    
new_lines = ['-DOCSTART- -X- -X- O\n\n']
for l in lines:
    if l!='\n':
        line = l.split('\t')
        new_line = line[0] + ' NN O ' + line[-1]
        new_lines.append(new_line)
    else:
        new_lines.append('\n')
    
new_conll_file = conll_file.replace(".tsv", "_4cols.csv")  

with open(new_conll_file, 'w') as f:
    f.write(''.join(new_lines))


In [67]:
cas_file = '/home/taille/these/data/corpus_dalloux/cas_BIO_val.csv'

cas_df = pd.read_csv(cas_file, sep='\t', names=['word', 'ner_tag'])

cas_df['pos_tag'] = 'NN'
cas_df['chunk_tag'] = 'O'
cas_df = cas_df[['word', 'pos_tag', 'chunk_tag', 'ner_tag']]
cas_df.loc[cas_df['word'].isna(), :] = ''

cas_df.to_csv('/home/taille/these/data/corpus_dalloux/cas_conll_val.csv', index=False, sep=' ')